# Exploratory Data Analysis
## States Table

In [1]:
# Go One Level Up to set the Workinkg Space
import os
os.chdir("..")

In [2]:
# Print current directory
os.getcwd()

'/Users/Rubenvargas/Documents/GitHub/NBA'

In [14]:
# Python Libreries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sql_queries import read_sql

In [4]:
# Read Data from the States Table
states_df = read_sql("""--sql
    SELECT * FROM states
""")

In [17]:
states_df.describe()

,year,personal_income,pipc,population
count,5487.000000,5.445000e+03,5445.000000,5.445000e+03
mean,1975.000000,1.545327e+05,15551.076584,7.454475e+06
std,26.847682,4.029676e+05,17703.579322,1.131203e+07
min,1929.000000,4.790000e+01,122.000000,9.000000e+04
25%,1952.000000,3.587900e+03,1645.000000,1.327287e+06
50%,1975.000000,2.216260e+04,6381.000000,3.309175e+06
75%,1998.000000,1.106388e+05,26600.000000,7.933000e+06
max,2021.000000,4.818998e+06,96873.000000,8.587219e+07


In [18]:
states_df.dtypes

state               object
year                 int64
personal_income    float64
pipc               float64
population         float64
dtype: object

states_df.dtypes

In [25]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
for col in states_df.select_dtypes(include=numerics).columns:
    fig = go.Figure(data = go.Histogram(x=states_df[col]))
    fig.layout.update(title=col)
    fig.show()

In [21]:
states_df.state.value_counts()

Alabama                 93
Rhode Island            93
New England             93
New Hampshire           93
New Jersey              93
New Mexico              93
New York                93
North Carolina          93
North Dakota            93
Ohio                    93
Oklahoma                93
Oregon                  93
Pennsylvania            93
Plains                  93
Rocky Mountain          93
Nebraska                93
South Carolina          93
South Dakota            93
Southeast               93
Southwest               93
Tennessee               93
Texas                   93
Utah                    93
Vermont                 93
Virginia                93
Washington              93
West Virginia           93
Wisconsin               93
Nevada                  93
Montana                 93
Alaska                  93
Idaho                   93
Arizona                 93
Arkansas                93
California              93
Colorado                93
Connecticut             93
D

Chequeo de los estados que no hacen match

In [26]:
from nba_api.stats.static import teams

In [30]:
nba_states = [st['state'] for st in teams.get_teams()]
[st for st in nba_states if st not in states_df.state.unique()]

['Atlanta', 'Ontario']

Estos no hacen match o porque no son estados de la unión americana (Ontario) o porque no son estados (Atlanta). En el último caso, hay que hacer un cambio en la ETL de la tabla de equipos para que el estado que se guarde sea el de Georgia, porque viene con ese error de la api de la NBA